In [1]:
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
mask = np.load(r"C:\Users\DU\aman_fastmri\Data\mask_4x_320_random.npy")  # Shape: (1, 320, 320)
print("og shape:", mask.shape)

# # Use np.tile to reshape it to (1, 320, 320, 1)
# # var_sampling_mask = np.tile(var_sampling_mask[..., np.newaxis], (1, 1, 1, 1))  # Final shape: (1, 320, 320, 1)
# mask = np.tile(mask, (1, 320, 1, 2))  # tile height=320 times

# # Confirm final shape
# print("New shape:", mask.shape) 
# mask_for_plot = np.squeeze(mask[...,0])  # Shape: (320, 320)

# # Plot
# plt.figure(figsize=(5, 5))
# plt.imshow(mask_for_plot, cmap='gray')
# plt.title("Tiled Sampling Mask (320x320)")
# plt.axis('off')
# plt.show()

og shape: (1, 1, 320, 1)


In [4]:
val_folder = r"D:\fastmri_singlecoil_FSSCAN\val_norm"
import h5py
import numpy as np
import glob
import os

kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))
half_val = len(kspace_files_list_val) 
half_val

199

In [5]:
%run ./model.ipynb

Model: "DSMENet_Functional"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 256, 256, 2  0           []                               
                                )]                                                                
                                                                                                  
 SRUN_1 (SRUN)                  ((None, 256, 256, 1  542204      ['input_image[0][0]']            
                                6),                                                               
                                 (None, 256, 256, 2                                               
                                ))                                                                
                                                                                 

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# ============================================================
# Directory Setup
# ============================================================
save_dir = "./checkpoints_dmse_full"
H, W = 320, 320
EPOCHS = 50
LEARNING_RATE = 1e-4
model = build_DSMENet_functional(
    N=6, M=1, T=2,
    H=320, W=320, C=2
)

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
model.compile(optimizer=optimizer)

#
# ============================================================
# Load Initial Weights (Optional Resume)
# ============================================================
if tf.train.latest_checkpoint(save_dir):
    model.load_weights(tf.train.latest_checkpoint(save_dir))
    print("✅ Loaded latest checkpoint")
else:
    print("ℹ️ No checkpoint found. Training from scratch.")

✅ Loaded latest checkpoint


In [9]:
import os
import numpy as np
import h5py
import glob
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# # Path to validation folder
# # val_folder = "F:/denoised_preprocessed_h5_val"

# val_folder = r"E:\fastmri\val_norm"
# # val_folder = r"D:\val_norm"

# # val_folder = r"G:\val_norm\val_norm"
# # files = sorted([os.path.join(val_folder, f) for f in os.listdir(val_folder) if f.endswith(".h5")])
# kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))
file_paths = kspace_files_list_val


# ----------------------
# HELPERS
# ----------------------
def to_complex(x):
    return x[..., 0] + 1j * x[..., 1]

def nmse(gt, pred):
    return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2 + 1e-10)

def compute_ssim(gt, pred, max_val):
    return structural_similarity(
        gt, pred,
        data_range=max_val,
        win_size=7,
        gaussian_weights=False,
        use_sample_covariance=False,
        K1=0.01,
        K2=0.03
    )

# ----------------------
# STORAGE
# ----------------------
ssim_list = []
psnr_list = []
nmse_list = []

# ----------------------
# PROCESSING
# ----------------------
for file in tqdm(kspace_files_list_val, desc="Processing volumes"):
    with h5py.File(file, 'r') as f:
        image_under = f["image_under"][:]   # (S, H, W, 2)
        image_full   = f["image_full"][:]     # (S, H, W, 2)
        max_val = float(f["max_val_full_image"][0])

    num_slices = image_under.shape[0]
        # Convert to complex
    # img_under_c = to_complex(image_under)
    # img_full_c  = to_complex(image_full)

    # # Magnitude
    # img_under_mag = np.abs(img_under_c)
    # img_full_mag  = np.abs(img_full_c)
    # img_full_mag = np.expand_dims(img_full_mag, axis=-1)


    # --------------------------------------------------
    # TILE MASK FOR THIS VOLUME
    # --------------------------------------------------
    #mask_batch = np.tile(mask, (num_slices, 1, 1, 1))

    # --------------------------------------------------
    # MODEL INFERENCE
    # --------------------------------------------------
    _, pred = model.predict(
        image_under,
        batch_size=1,
        verbose=0
    )
    #print("pred",pred.shape)
    #print("img_full_mag",img_full_mag.shape)
    

    #max_val=img_full_mag.max()
    image_full *= max_val
    pred*= max_val
    # pred *= max_val  # Scale predicted output to original intensity range
    psnr_val = peak_signal_noise_ratio(image_full, pred, data_range=max_val)

    # # Convert to complex and get magnitude
    gt_mag = np.abs(to_complex(image_full))
    pred_mag = np.abs(to_complex(pred))

    # Volume-wise PSNR and NMSE
    #psnr_val = peak_signal_noise_ratio(gt_mag,pred_mag, data_range=max_val)
    nmse_val = nmse(gt_mag.flatten(), pred_mag.flatten())

    psnr_list.append(psnr_val)
    nmse_list.append(nmse_val)

    # Slice-wise SSIM
    # Slice-wise SSIM
    for i in range(gt_mag.shape[0]):
        ssim_val = compute_ssim(gt_mag[i], pred_mag[i], max_val)
        ssim_list.append(ssim_val)
# ----------------------
# REPORT
# ----------------------
print("\n" + "=" * 40)
print(f"PSNR : {np.mean(psnr_list):.4f} ± {np.std(psnr_list):.4f} dB")
print(f"NMSE (Mag, volume): {np.mean(nmse_list):.6f} ± {np.std(nmse_list):.6f}")
print(f"SSIM (Mag, slice):  {np.mean(ssim_list):.4f} ± {np.std(ssim_list):.4f}")

print("=" * 40)


Processing volumes: 100%|████████████████████████████████████████████████████████████| 199/199 [34:38<00:00, 10.44s/it]


PSNR : -29.5519 ± 2.6771 dB
NMSE (Mag, volume): 77742.533720 ± 24906.322468
SSIM (Mag, slice):  0.0000 ± 0.0000
